# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to use [requests](http://www.python-requests.org/en/master/) to download HTML pages from a website?
* How to select content on a webpage with [lxml](http://lxml.de/)? 

You can either use Spark DataFrame or [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) to do the assignment. In comparison, pandas.DataFrame has richer APIs, but is not good at distributed computing.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of HTML, DOM, and XPath. I found that this is a good resource: [https://data-lessons.github.io](https://data-lessons.github.io/library-webscraping-DEPRECATED/). Please take a look at

* [Selecting content on a web page with XPath
](https://data-lessons.github.io/library-webscraping-DEPRECATED/xpath/)
* [Web scraping using Python: requests and lxml](https://data-lessons.github.io/library-webscraping-DEPRECATED/04-lxml/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. One day, you want to analyze CS faculty data and answer two interesting questions:

1. Who are the CS faculty members?
2. What are their research interests?

To do so, the first thing is to figure out what data to collect.

## Task 1: SFU CS Faculty Members

You find that there is a web page in the CS school website, which lists all the faculty members as well as their basic information. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawling Web Page

A web page is essentially a file stored in a remote machine (called web server). You can use [requests](http://www.python-requests.org/en/master/) to open such a file and read data from it. Please complete the following code to download the HTML page and save it as a text file (like [this](./faculty.txt)). 

In [1]:
# 1. Download the webpage
import requests
response = requests.get('https://www.sfu.ca/computing/people/faculty.html')
# 2. Save it as a text file (named faculty.txt)
with open("faculty.txt",'w') as f: 
    f.write(response.text) 

### (b) Extracting Structured Data

An HTML page follows the Document Object Model (DOM). It models an HTML page as a tree structure wherein each node is an object representing a part of the page. The nodes can be searched and extracted programmatically using XPath. Please complete the following code to transform the above HTML page to a CSV file (like [this](./faculty_table.csv)). 

In [2]:
import lxml.html 

# 1. Open faculty.txt
with open('faculty.txt','r') as myfile:
    file = myfile.read()
    
# 2. Parse the HTML page as a tree structure
tree = lxml.html.fromstring(file)

# 3. Extract related content from the tree using XPath
tree_name_rank = tree.xpath('//div[@class = "text"]//h4')
name = []
rank = []
for x in tree_name_rank:
    text_name_rank = str(x.text)
    text_name_rank = text_name_rank.strip()
    if text_name_rank == 'None':
        continue
    name_rank = text_name_rank.split(',')
    if "Director" not in name_rank[0]: 
        name.append(name_rank[0])
        rank.append(name_rank[1])
        
import pandas as pd
 
df = pd.DataFrame({'name':name})
df['rank'] = rank

tree_area = tree.xpath('//div[@class = "text"]')
area = []
for x in tree_area:
    for y in x:
       var1 = str(y.text_content())   
       if (var1.startswith(('Area','Fred'))):  
        var2 = var1.replace('Area:', '')
        var2 = var2.replace(';',',')
        var2 = var2.strip()
        if not var1.startswith('Fred'):
         area.append(var2)
        else:
         area.append(var2[28:])   
df['area'] = area

homepage = 'http://www.sfu.ca'
tree_link = tree.xpath("//div[@class = 'text']//a")
profile = []
home_link = [None] * 60
for x in tree_link:
     var = x.text_content()
     var = var.strip()   
     if var == "Profile & Contact Information":
        link = x.get('href')
        if not link.startswith(homepage): 
             link = homepage + link
        profile.append(link)  
        
count = 0    
for x in tree_link:    
    var = x.text_content()
    var = var.strip()
    if var == "Profile & Contact Information":
     count = count + 1   
    if var == "Home Page":
     link_home = x.get('href')
     link_home = link_home.strip()
     home_link[count-1]=link_home
     
count = 0
for x in home_link:
    if x is None:
        home_link[count]=""
    count = count + 1    

df['profile'] = profile
df['homepage'] = home_link

df.set_index('name',inplace = True)

# 4. Save the extracted content as a csv file (named faculty_table.csv)
df.to_csv("faculty_table.csv")


## Task 2: Research Interests

Suppose you want to know the research interests of each faculty. However, the above crawled web page does not contain such information. 

### (a) Crawling Web Page

You notice that such information can be found on the profile page of each faculty. For example, you can find the research interests of Dr. Jiannan Wang from [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please complete the following code to download the profile pages and save them as text files. There are 60 faculties, so you need to download 60 web pages in total. 

In [3]:
# 1. Download the profile pages of 60 faculties
# 2. Save each page as a text file
import requests
count = 1
for pages in profile:
    r = requests.get(pages)
    with open(str(count) +'.txt','w') as f: 
     f.write(r.text) 
     count = count + 1

### (b) Extracting Structured Data

Please complete the following code to extract the research interests of each faculty, and generate a file like [this](./faculty_more_table.csv). 

In [4]:
import lxml.html
import requests
import pandas as pd
research_interests = []
c = 1
list_number = []
while c < 61:
    list_number.append(c)
    c = c + 1
# 1. Open each text file and parse it as a tree structure
for x in list_number:  
    interests = []
    with open(str(x) +'.txt','r') as myfile:
        file = myfile.read()    
    tree = lxml.html.fromstring(file)
# 2. Extract the research interests from each tree using XPath
    tree_element = tree.xpath("//div[@class = 'ruled']")
    if not tree_element:
      tree_element = tree.xpath("//div[@class = ' listed' or @class = 'listed']")
    flag = 0
    for x in tree_element:
      for y in x: 
        if flag == 1:
            interests.append(y.text_content().strip().split('\n'))
            flag = 0;
        if str(y.text).strip().lower() == "research interests":
            flag = 1    
    interests_final = []
    interests_str = str(interests)
    interests_str = interests_str.replace('\\xa0', '')
    interests_str = interests_str.replace('[', '')
    interests_str = interests_str.replace(']', '')
    interests_str = interests_str.replace("'',", "")
    interests_str = "[" + interests_str + "]"
    interests_str = interests_str.strip()
    research_interests.append(interests_str)

# 3. Add the extracted content to faculty_table.csv    
df['research_interests'] = research_interests
#(PLEASE RUN ALL THE CELLS) (PLEASE RUN ALL THE CELLS) (PLEASE RUN ALL THE CELLS)
# If you choose to run this cell only please uncomment the steps given below.
# Also make sure to comment out df['research_interests'] = research_interests above.
#one can also read the csv file as a dataframe and then add the research_interests column
#this can be done as follows:
#df = pd.read_csv('faculty_table.csv')
#df['research_interests'] = research_interests
#df.set_index('name',inplace = True)

# 4. Generate a new CSV file, named faculty_more_table.csv
df.to_csv("faculty_more_table.csv")

## Submission

Complete the code in this [notebook](A1.ipynb), and submit it to the CourSys activity `Assignment 1`.